In [1]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import RandomFlip, RandomRotation, Dense, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# Define paths and load image names
images_fp = '/content/images (1).tar.gz'  # Update this path
image_names = [os.path.basename(file) for file in glob.glob(os.path.join(images_fp, '*.jpg'))]

# Define labels and encode them
labels = [' '.join(name.split('_')[:-1]) for name in image_names]

def label_encode(label):
    label_mapping = {
        'Abyssinian': 0, 'Bengal': 1, 'Birman': 2, 'Bombay': 3, 'British Shorthair': 4,
        'Egyptian Mau': 5, 'american bulldog': 6, 'american pitbull terrier': 7,
        'basset hound': 8, 'beagle': 9, 'boxer': 10, 'chihuahua': 11,
        'english cocker spaniel': 12, 'english setter': 13, 'german shorthaired': 14,
        'great pyrenees': 15
    }
    return label_mapping.get(label, None)

# Load and preprocess images
features, labels_encoded = [], []
image_size = (224, 224)

for img_name, label in zip(image_names, labels):
    img_path = os.path.join(images_fp, img_name)
    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)  # Preprocess for ResNet50
    features.append(img_array)
    labels_encoded.append(label_encode(label))

# Convert lists to numpy arrays
features = np.array(features)
labels_encoded = np.array(labels_encoded)

# One-hot encode the labels
num_classes = len(set(labels_encoded))
labels_one_hot = tf.keras.utils.to_categorical(labels_encoded, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_one_hot, test_size=0.2, random_state=42)

# Data augmentation
data_augmentation = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.2),
])

# Define the model
def build_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False  # Freeze the base model
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

# Build the model
input_shape = (224, 224, 3)
model = build_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Plot training history
def plot_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()

    plt.show()

plot_history(history)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4f}')

ValueError: zero-size array to reduction operation maximum which has no identity